### 接受任意参数函数
注意,一个*参数只能出现在函数定义中最后一个位置,**参数只能出现在最后一个参数.*参数前也可以定义其他参数.单个/表示/前面的为位置参数,单个*表示后面为字典参数

In [8]:
def func_a(x,/,**kargs):
    pass

# key在*后面表示只接受,关键字参数
def func_b(x,*,key):
    pass

# func_a(x=1,**{'1':2}) required positional argument
func_b(1,key=2)

def func_c(*arg,**kargs):
    print(type(arg),type(kargs))

func_c()

<class 'tuple'> <class 'dict'>


### 返回多个值的函数

In [ ]:
# 返回是一个元组
def func():
    return 1,2,3

### 定义匿名函数
使用lambda定义简单的匿名函数,lambda使用有限制,只能指定单个表达式,不能包含其他语言特性(if..else..;while)

In [9]:
strings=['123','1','2345']
# sored 非原地会返回一个新的排序的数组
new=sorted(strings,key=lambda x:len(x))

### 匿名函数在定义时捕获到值

In [12]:
x=10
func=lambda y:x+y
func_=lambda y,x=x:x+y
x=20
funb=lambda y:x+y
print(func(10)) # 30 
print(funb(10)) # 30
# x是自由变量,在运行才进行绑定.要想使用必须定义成默认参数

funcs=[lambda n:n+i for i in range(5)]
# i是存放的最后一个值
for fun in funcs:
    print(fun(0))

30
30
4
4
4
4
4


### 减少调用对象参数个数
functools.partial()
> partial 不完整

In [2]:
from functools import partial

def many(a,b,c,d,e):
    print(a,b,c,d,e)

one = partial(many,1,2,3,4)

one(1)

1 2 3 4 1


### 带额外状态信息的回调函数
- 使用一个ResultHandler类
- 使用必包捕获状态
- 使用携程的send

In [19]:
def apply_async(func,args,*,callback):
    result=func(*args)
    callback(result)

def print_result(args):
    print(f'{args}')

def add(x,y):
    return x+y


apply_async(add,(1,2),callback=print_result)

# 使用类来扩展信息
class ResultHandler:
    def __init__(self,i) -> None:
        self.i=i
    
    def handler(self,result):
        self.i+=1
        print(f'[{self.i}] {result}')
handler=ResultHandler(1)
apply_async(add,(2,2),callback=handler.handler)

# 使用必包来扩展信息
def make_handler():
    i=1
    def wrapper(args):
       nonlocal i
       i+=1
       print(f'[{i}] {args}') 

    return wrapper

apply_async(add,(2,2),callback=make_handler())

#todo 使用协程

3
[2] 4
[2] 4


### 利用必包将单个类转换为函数

In [ ]:
from urllib.request import urlopen

class UrlTemplate:
    def __init__(self,template):
        self.template=template
    
    def open(self,**kargs):
        return urlopen(self.template.format_map(kargs))

baidu=UrlTemplate('www.baidu.com?s={name}')
baidu.open(name='name')

def open(template):
    def wrapper(**kargs):
        return urlopen(template.format_map(kargs))
    return wrapper

### 访问闭包中定义的变量
通过在闭包定义get,set方法然后再把这个方法,赋值到返回的functon对象上

In [3]:
def simple():
    i=0
    def wrapper():
        nonlocal i
        i+=1
        print(i)

    def set_i(value):
        nonlocal i
        i=value

    def get_i():
        nonlocal i
        return i

    wrapper.get_i=get_i
    wrapper.set_i=set_i
    return wrapper
    
func=simple()
func()

func.set_i(2)
print(func.get_i())


1
2


In [21]:
#  利用sys._getframe(1).f_locals() 获取上一个栈中的方法
import sys

class ClosureInstance:
    def __init__(self,local=None):
        if not local:
            local=sys._getframe(1).f_locals #dic
            # update dict
        self.__dict__.update((key,val)for key,val in local.items() if callable(val))


    
    def __len__(self):
        return self.__dict__['__len__']
    

def func():
    items=[]
    def push(item):
        items.append(item)
    
    def pop():
        return items.pop()
    
    return ClosureInstance()

stack=func()

stack.push(1)
stack.pop()

1